In [87]:
from pymongo import MongoClient
from pymongo.errors import PyMongoError

class AnimalShelter:
    def __init__(self,
                 user: str,
                 password: str,
                 host: str = "localhost",
                 port: int = 27017,
                 auth_db: str = "animal_shelter"  
                 ):
        """
        Connect to MongoDB and select the 'animal_shelter' database.
        If your user was created in another DB (e.g. 'admin'), pass auth_db="admin".
        """
        uri = f"mongodb://localhost:27017"
        try:
            self.client = MongoClient(uri)
            self.database = self.client["animal_shelter"]
            print(f"✔ Connected to MongoDB (authSource={auth_db})")
        except PyMongoError as e:
            print("✘ Connection error:", e)
            raise

    def create(self, data: dict) -> bool:
        """Inserts one document. Returns True on success, False otherwise, with debug prints."""
        # 1) Input sanity check
        if not isinstance(data, dict) or not data:
            print("🔴 create(): Bad input — you must pass a non‑empty dict")
            return False

        # 2) Echo document contents
        print("🔄 create(): attempting to insert this document:")
        for k, v in data.items():
            print(f"    • {k!r}: {v!r}")

        # 3) Attempt insert
        try:
            result = self.database.animals.insert_one(data)
            print(f"✅ create(): acknowledged={result.acknowledged}")
            if result.acknowledged:
                print(f"    • new document _id = {result.inserted_id}")
                return True
            else:
                print("🔴 create(): insert was not acknowledged")
                return False
        except Exception as e:
            print("🔴 create(): Exception during insert_one:")
            print(f"    • {type(e).__name__}: {e}")
            return False

    def read(self, query: dict) -> list:
        """Finds all matching docs (hides _id). Always returns a list (possibly empty)."""
        if not isinstance(query, dict):
            raise ValueError("read() requires a dict query")
        try:
            return list(self.database.animals.find(query, {"_id": False}))
        except Exception as e:
            print("🔴 read(): Exception during find():", e)
            return []

    def update(self,
               query: dict,
               update_values: dict,
               multi: bool = False
               ) -> int:
        """
        Updates matching docs.
        If multi=False → update_one; if multi=True → update_many.
        Returns the number of documents modified.
        """
        if not query or not update_values:
            raise ValueError("update() needs both query and update_values")
        try:
            if multi:
                result = self.database.animals.update_many(query, update_values)
            else:
                result = self.database.animals.update_one(query, update_values)
            print(f"✅ update(): matched={result.matched_count}, modified={result.modified_count}")
            return result.modified_count
        except Exception as e:
            print("🔴 update(): Exception during update:", e)
            return 0

    def delete(self,
               query: dict,
               multi: bool = False
               ) -> int:
        """
        Deletes matching docs.
        If multi=False → delete_one; if multi=True → delete_many.
        Returns the number of documents deleted.
        """
        if not isinstance(query, dict) or not query:
            raise ValueError("delete() requires a non‑empty dict query")
        try:
            if multi:
                result = self.database.animals.delete_many(query)
            else:
                result = self.database.animals.delete_one(query)
            print(f"✅ delete(): deleted={result.deleted_count}")
            return result.deleted_count
        except Exception as e:
            print("🔴 delete(): Exception during delete:", e)
            return 0


# -----------------------
# Example usage
# -----------------------
if __name__ == "__main__":
    username = "aacuser"
    password = "Gaccurser"
    # If your user was created in 'admin', pass auth_db="admin"
    shelter = AnimalShelter(username, password, auth_db="")

    # 1) Create
    valid_animal = {
        'age_upon_outcome': "4 years",
        'animal_id': "A000001",
        'animal_type': "Cat",
        'breed': "Domestic Shorthair Mix",
        'color': "Black Tabby",
        'date_of_birth': "2018-03-27",
        'datetime': "2018-03-27 11:14:00",
        'monthyear': "2018-03-27T11:14:00",
        'name': "Binx",
        'outcome_subtype': "",
        'outcome_type': "Adoption",
        'sex_upon_outcome': "Spayed Male",
        'location_lat': 30.6525984560228,
        'location_long': -97.7419963476444,
        'age_upon_outcome_in_weeks': 52.92
    }
    print("Create Result:", shelter.create(valid_animal))

    # 2) Read
    results = shelter.read({"name": "Binx"})
    print("Read Result:", results)

    # 3) Update
    modified = shelter.update(
        {"animal_id": "A000001"},
        {"$set": {"outcome_type": "Return to Owner"}},
        multi=False
    )
    print("Modified Count:", modified)

    # 4) Delete
    deleted = shelter.delete({"animal_id": "A000001"}, multi=False)
    print("Deleted Count:", deleted)
if __name__ == "__main__":
    username = "aacuser"
    password = "Gaccurser"
    shelter = AnimalShelter(username, password, auth_db="")

    # Multiple test entries for filtering
    sample_animals = [
        {
            'age_upon_outcome': "1 year",
            'animal_id': "A000002",
            'animal_type': "Dog",
            'breed': "Labrador Retriever Mix",
            'color': "Black",
            'date_of_birth': "2022-04-12",
            'datetime': "2023-05-10 14:00:00",
            'monthyear': "2023-05",
            'name': "Rocky",
            'outcome_subtype': "Foster",
            'outcome_type': "Adoption",
            'sex_upon_outcome': "Intact Female",
            'location_lat': 30.2672,
            'location_long': -97.7431,
            'age_upon_outcome_in_weeks': 52
        },
        {
            'age_upon_outcome': "6 months",
            'animal_id': "A000003",
            'animal_type': "Dog",
            'breed': "German Shepherd",
            'color': "Tan",
            'date_of_birth': "2023-01-10",
            'datetime': "2023-07-10 09:30:00",
            'monthyear': "2023-07",
            'name': "Zeus",
            'outcome_subtype': "",
            'outcome_type': "Transfer",
            'sex_upon_outcome': "Intact Male",
            'location_lat': 30.3000,
            'location_long': -97.7000,
            'age_upon_outcome_in_weeks': 26
        },
        {
            'age_upon_outcome': "2 years",
            'animal_id': "A000004",
            'animal_type': "Dog",
            'breed': "Doberman Pinscher",
            'color': "Brown",
            'date_of_birth': "2021-04-01",
            'datetime': "2023-03-20 16:00:00",
            'monthyear': "2023-03",
            'name': "Nina",
            'outcome_subtype': "",
            'outcome_type': "Adoption",
            'sex_upon_outcome': "Intact Male",
            'location_lat': 30.2800,
            'location_long': -97.7600,
            'age_upon_outcome_in_weeks': 104
        },
        {
            'age_upon_outcome': "3 years",
            'animal_id': "A000005",
            'animal_type': "Cat",
            'breed': "Domestic Shorthair Mix",
            'color': "White",
            'date_of_birth': "2020-08-05",
            'datetime': "2023-08-05 10:15:00",
            'monthyear': "2023-08",
            'name': "Luna",
            'outcome_subtype': "Euthanasia",
            'outcome_type': "Euthanasia",
            'sex_upon_outcome': "Spayed Female",
            'location_lat': 30.3100,
            'location_long': -97.7500,
            'age_upon_outcome_in_weeks': 156
        },
        {
            'age_upon_outcome': "1 year",
            'animal_id': "A000006",
            'animal_type': "Dog",
            'breed': "Siberian Husky",
            'color': "Grey",
            'date_of_birth': "2022-06-15",
            'datetime': "2023-06-20 11:45:00",
            'monthyear': "2023-06",
            'name': "Shadow",
            'outcome_subtype': "",
            'outcome_type': "Return to Owner",
            'sex_upon_outcome': "Intact Male",
            'location_lat': 30.2500,
            'location_long': -97.7100,
            'age_upon_outcome_in_weeks': 52
        }
    ]

    for animal in sample_animals:
        print("Create Result:", shelter.create(animal))

    # Read back to verify
    print("All Dogs Retrieved:", shelter.read({"animal_type": "Dog"}))



✔ Connected to MongoDB (authSource=)
🔄 create(): attempting to insert this document:
    • 'age_upon_outcome': '4 years'
    • 'animal_id': 'A000001'
    • 'animal_type': 'Cat'
    • 'breed': 'Domestic Shorthair Mix'
    • 'color': 'Black Tabby'
    • 'date_of_birth': '2018-03-27'
    • 'datetime': '2018-03-27 11:14:00'
    • 'monthyear': '2018-03-27T11:14:00'
    • 'name': 'Binx'
    • 'outcome_subtype': ''
    • 'outcome_type': 'Adoption'
    • 'sex_upon_outcome': 'Spayed Male'
    • 'location_lat': 30.6525984560228
    • 'location_long': -97.7419963476444
    • 'age_upon_outcome_in_weeks': 52.92
✅ create(): acknowledged=True
    • new document _id = 68055dc9e8fd8156919e02b5
Create Result: True
Read Result: [{'age_upon_outcome': '4 years', 'animal_id': 'A000001', 'animal_type': 'Cat', 'breed': 'Domestic Shorthair Mix', 'color': 'Black Tabby', 'date_of_birth': '2018-03-27', 'datetime': '2018-03-27 11:14:00', 'monthyear': '2018-03-27T11:14:00', 'name': 'Binx', 'outcome_subtype': '', 'o